In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as gbm
import sklearn.svm as SVC

/Users/sarvesh/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Getting Data
train = pd.read_csv("train_set.csv")
train_labels = pd.read_csv("train_set_labels.csv")

train = pd.merge(train, train_labels, on='id')
train, test = train_test_split(train, test_size=0.2)


In [4]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
48129,64916,500.0,2011-04-04,Danida,1884,DWE,34.526311,-9.183715,Kwa Musa Mwilongo,0,...,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional
37400,60742,0.0,2011-03-19,Kibaha Town Council,65,DAWASCO,38.838296,-6.727160,Misugusugu Zahanati,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
40514,17996,0.0,2011-03-22,Government Of Tanzania,1552,DWE,38.372253,-4.647711,Kwa Mzee Zuberisarehe,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional
2695,49228,0.0,2013-01-23,Ministry Of Water,1188,DWE,33.465439,-2.084277,Kwa Bernad Kaliae,0,...,soft,good,dry,dry,lake,river/lake,surface,communal standpipe multiple,communal standpipe,non functional
24909,37470,20.0,2011-02-26,Private,-20,Priva,38.988658,-7.952749,Msikitini,0,...,soft,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,functional


In [5]:
test.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
22560,1067,0.0,2012-11-05,Government Of Tanzania,0,Government,33.350618,-3.730953,Kwa Mashema Mangasa,0,...,soft,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
13933,50520,500.0,2011-03-16,Dhv,295,DWE,36.053847,-8.885367,Kwa Lyampawe,0,...,coloured,colored,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional
31409,13262,0.0,2013-02-02,Wvt,1330,WVT,33.996505,-2.994830,Shule Ya Msingi Ng'Wang'Wita A,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
39700,70684,0.0,2011-07-20,Hesawa,0,DWE,31.737788,-1.427196,Kwa Raulian Mabugo,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional needs repair
5336,10572,0.0,2011-04-03,Cpps,0,Commu,35.714142,-6.230243,Kwa Yoram,0,...,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other,non functional


In [91]:
columns = list(set(list(train.columns)[1:-1]) - set(['num_private', 'basin', 
                                                     'lga', 'region', 'funder', 
                                                     'installer', 'scheme_name',
                                                    'scheme_management']))
X = train[columns]
Y = train[list(train.columns)[-1]]

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_extraction import DictVectorizer

from sklearn.preprocessing import LabelEncoder

X_p = X
lb_make = LabelEncoder()
for column in columns:
    if X[column].dtype not in ['int64', 'float64']:
        X_p[column] = X[column].astype('category')
        X_p[column] = lb_make.fit_transform(X[column].fillna(method='ffill'))
        
Y_p = lb_make.fit_transform(Y)
# ohe = OneHotEncoder()
# ohe.fit(X['funder'])
# ohe.transform(X['funder']).head()
# X.head()
# cols = list(set(columns) - set(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'region_code', 'district_code', 'population']))
# X_p = pd.get_dummies(X)


/Users/sarvesh/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/sarvesh/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [51]:
# from sklearn.feature_selection import SelectKBest
# skb = SelectKBest(k = 10000)
# skb.fit(X_p, Y)
# X_p = skb.transform(X_p, Y)
X_p.shape
X_p.head()

,ward,source,payment,public_meeting,recorded_by,region_code,waterpoint_type_group,longitude,waterpoint_type,extraction_type_group,...,payment_type,quantity_group,subvillage,source_class,management,source_type,permit,date_recorded,amount_tsh,construction_year
41534,748,8,6,1,0,12,1,33.837777,1,1,...,6,1,13020,0,8,6,0,63,0.0,0
15618,1497,8,0,1,0,12,1,33.835122,1,1,...,2,0,2957,0,7,6,2,109,0.0,0
35167,1575,3,0,0,0,14,1,33.227223,1,4,...,2,1,4219,0,7,0,2,275,0.0,0
20823,704,7,0,1,0,80,5,39.282792,6,6,...,2,1,15890,0,7,5,2,289,0.0,1975
52930,699,8,6,1,0,2,1,36.994883,1,1,...,6,2,2280,0,7,6,2,322,0.0,2009


In [93]:
# TODO
pca = PCA(n_components=18)
X_p = pca.fit_transform(X_p, Y)
# y = pca.fit_transform(Y_p)
# y.shape
X_p.shape

(47520, 18)

In [57]:
xgb_x = xgb.DMatrix(X_p)

In [94]:
dtrain = xgb.DMatrix(X_p, label=Y_p)

In [76]:
print(dtrain)

In [95]:
param = {'max_depth': 2, 'eta': 1, 'silent': 1}
param['nthread'] = 4
param['eval_metric'] = 'auc'
param['eval_metric'] = ['auc', 'ams@0']
bst = xgb.train(params=param, dtrain=dtrain)

In [83]:
bst.predict(X_p)

AttributeError: 'numpy.ndarray' object has no attribute 'feature_names'